In [97]:
import math
import copy
import random
import numpy as np

# Convert a rank id into a list of d-dimensional coordinates
def get_coord_from_id(id, dimensions):
    if len(dimensions) == 1:
        return [id]
    elif len(dimensions) == 2:
        return [id // dimensions[1], id % dimensions[1]]
    elif len(dimensions) == 3:
        return [(id // dimensions[1]) % dimensions[0], id % dimensions[1], id // (dimensions[0]*dimensions[1])]

# Convert d-dimensional coordinates into a rank id
def get_id_from_coord(coord, dimensions):
    if len(dimensions) == 1:
        return coord[0]
    elif len(dimensions) == 2:
        return coord[0]*dimensions[1] + coord[1]
    else:
        return int(coord[2]*(dimensions[0]*dimensions[1]) + get_id_from_coord(coord, dimensions[0:2]))

# Starting from the data of each node, computes the final data that we would expect after running an allreduce
def compute_expected_data_allreduce(data, p):
    expected_res = np.zeros(p)
    # Compute expected result (Each row is a host)
    for i in range(0, p):
        for j in range(0, p):
            expected_res[i] += data[j][i]
    return expected_res

# Gets the distance of the peer of rank 'id' at the step-th step on a dimension-dimensional network, considering the next_direction
def get_distance(id, step, next_direction, dimensions):
    distances = [1, 1, 3, 5, 11, 21, 43, 85] # Jacobsthal sequence!! (https://oeis.org/search?q=1%2C+1%2C+3%2C+5%2C+11%2C+21&language=english&go=Search)
    dim = step % len(dimensions)
    step_relative_to_dim = step // len(dimensions)
    return (next_direction[id][dim])*distances[step_relative_to_dim]

# Gets the peer of rank 'id' at the step-th step on a dimension-dimensional network, considering the next_direction
def get_peer(id, step, next_direction, dimensions):
    distance = get_distance(id, step, next_direction, dimensions)
    coord = get_coord_from_id(id, dimensions)
    target_dim =  step % len(dimensions) # TODO: Extend for the cases where we start from different dimensions (multiported)
    peer_coord = coord
    peer_coord[target_dim] = (peer_coord[target_dim] + distance) % dimensions[target_dim]
    return int(get_id_from_coord(peer_coord, dimensions))

def recv(receiver, sender, data_new, data_old):
    # Aggregate
    for k in range(0, len(data_new[receiver])):
        data_new[receiver][k] += data_old[sender][k]

In [98]:
# Algo:
# 1. I take the tuple representing the coordinates
# 2. Each node proceeds one dimension at a time (in a synchronized way, e.g. x first, then y, then z)
# 3. Once a dimension is decided, the decision is whether to go on the positive or negative direction. 
#    Let us suppose that in the first step everyone picks the x dimension. Even nodes on that dimension
#    will send towards the positive direction, odd nodes towards the negative direction.

dimensions = [32, 32]
#dimensions = [16]
p = 1
for d in dimensions:
    p *= d

data = np.zeros((p, p))
next_direction = np.zeros((p, len(dimensions))) # For each dimension, each rank remembers on which direction it should send next time
sum_of_distances = 0
sum_of_distances_ref = 0
bw_new = 0
bw_ref = 0
bw_ideal = 0

# Set starting directions
for i in range(0, p):
    coord = get_coord_from_id(i, dimensions)
    for c in range(0, len(coord)):
        if coord[c] % 2 == 0:
            next_direction[i][c] = 1
        else:
            next_direction[i][c] = -1

# Initialize random starting data
for i in range(0, p):
    for j in range(0, p):
        data[i][j] = random.randint(0, 100)

expected_res = compute_expected_data_allreduce(data, p)

for step in range(0, int(math.log2(p))):        
    data_c = copy.deepcopy(data)
    # Bit array to check that no one sends the same data to more than one node
    recv_from = np.zeros(p)
    for rank in range(0, p):                   
        peer = get_peer(rank, step, next_direction, dimensions)         
        next_direction[rank][step % len(dimensions)] *= -1
        # Check that each node sends at most at one node
        if recv_from[peer]:
            print("Double recv")
            exit(-1)
        #print("Step " + str(i) + ": " + str(j) + "->" + str(peer))
        recv_from[peer] = 1
        recv(rank, peer, data_c, data)
    # Check that everyone sent something
    for k in range(0, p):
        if not recv_from[k]:
            print("Nosent")
            exit(-1)

    # Compute some stats
    distance = abs(get_distance(0, step, next_direction, dimensions))
    sum_of_distances += distance         
    bw_ideal += (1/2**(step+1)) 
    bw_new += (1/2**(step+1))*distance
    if len(dimensions) == 1:
        sum_of_distances_ref += 2**step
        bw_ref += (1/2**(step+1))*2**step
    elif len(dimensions) == 2:          
        sum_of_distances_ref += 2**math.floor(step/2)
        bw_ref += (1/2**(step+1))*2**math.floor(step/2)
    elif len(dimensions) == 3:
        sum_of_distances_ref += 2**math.floor(step/3)
        bw_ref += (1/2**(step+1))*2**math.floor(step/3)
    data = copy.deepcopy(data_c)
# Check correctness of the result
fail = False
for i in range(0, p):
    for j in range(0, p):
        if data[i][j] != expected_res[j]:
            print("FAIL!")
            exit(1)
#print(data)
#print(expected_res)
print("Sum of distances: " + str(sum_of_distances) + " ref: " + str(sum_of_distances_ref))
print("Bw new: " + str(bw_new) + " bw ref: " + str(bw_ref) + " bw ideal: " + str(bw_ideal))


Sum of distances: 42.0 ref: 62
Bw new: 1.1689453125 bw ref: 1.453125 bw ideal: 0.9990234375


In [194]:
import copy
import math
p = 32
steps = int(math.log2(p))
d = [1, 1, 3, 5, 11, 21, 43, 85]
for starting_step in range(0, steps):
    a = [-1]*p
    root = 0 #len(a) // 2
    for step in range(starting_step, steps):
        if (root % 2 == 0 and step % 2 == 0) or (root % 2 != 0 and step % 2 != 0):
            dest = (root + d[step]) % len(a)
        else:
            dest = (root - d[step]) % len(a)
        a[dest] = step
        root = dest
        if step != steps - 1:
            for x in range(0, len(a)):
                if a[x] <= step and a[x] != -1:
                    if (x % 2 == 0 and (step + 1) % 2 == 0) or (x % 2 != 0 and (step + 1) % 2 != 0):
                        dest = (x + d[step + 1]) % len(a)                    
                    else:
                        dest = (x - d[step + 1]) % len(a)
                    a[dest] = step + 1
    a = np.array(a)
    a[a >= 0] = 1
    a[a == -1] = 0
    print("Step " + str(starting_step) + ": " + str(a))

Step 0: [0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0]
Step 1: [0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 1]
Step 2: [0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0]
Step 3: [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
